# Thermal Expansion and Aluminum Bussing

Mallory Moxham // Nov 30, 2024

Add Calculations for spring washers

### Constants and Set-up Parameters

In [27]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt



####### TEMPERATURE CONDITIONS
T0 = 15                             # C - minimum expected temperature
Tmax = 60                           # C - maximum acceptable and expected temperature
dT_max = Tmax - T0                  # C - maximum delta T expected




####### BOLTS
# Create a bolt class
class bolt_size:
    # Class variables
    E = 200 * 10**3             # N/mm2 - bolt young's modulus
    Sp = 600                    # N/mm2 - Class 8.8 Bolt Proof Strength
    wt = 1.5                    # mm - washer thickness
    CTE = 12.3 * 10**(-6)       # C-1 - Linear CTE for Alloy Steel

    # Class Item Definition
    def __init__(self, d, md, w_OD, At):
        self.d = d              # mm - major diameter
        self.md = md            # mm - minor diameter
        self.w_OD = w_OD        # mm - washer outer diameter
        self.At = At            # mm2 - area in tension, Table 8-1 Shigley
        self.Ax = math.pi / 4 * d**2    # mm2 - area based on major diameter

# 1) M6
M6_d = 6                        # mm - major diameter
M6_md = 4.917                   # mm - minor diameter
M6_w_OD = 12                    # mm - washer outer diameter
M6_At = 20.1                    # mm2 - area in tension, Table 8-1 Shigley
M6 = bolt_size(d = M6_d, md = M6_md, w_OD = M6_w_OD, At = M6_At)

# 2) M8
M8_d = 8                        # mm - major diameter
M8_md = 6.647                   # mm - minor diameter
M8_w_OD = 16                    # mm - washer outer diameter
M8_At = 36.6                    # mm2 - area in tension, Table 8-1 Shigley
M8 = bolt_size(d = M8_d, md = M8_md, w_OD = M8_w_OD, At = M8_At)

# 3) M10 (for Sensata Contactors)
M10_d = 10                      # mm - major diameter
M10_md = 8.376                  # mm - minor diameter
M10_w_OD = 20                   # mm - washer outer diameter
M10_At = 58                     # mm2 - area in tension, Table 8-1 Shigley
M10 = bolt_size(d = M10_d, md = M10_md, w_OD = M10_w_OD, At = M10_At)




####### COPPER COMPONENTS
Cu_CTE = 16.7 * 10**(-6)        # C-1 - Linear CTE for copper
Cu_E = 110 * 10**3              # N/mm2 - Young's Modulus of copper

# 1) Copper Lug - uses an M6 mating part
lug_L0 = 2.02                   # mm - initial thickness
lug_holeD = 7                   # mm - hole diameter on lug
lug_Ax = math.pi / 4 * (M6.w_OD**2 - lug_holeD**2)  # mm2 - net area of lug

# 2) E-meter Copper Parts
EM_L0 = 8                       # mm - initial thickness
EM_holeD = 8                    # mm - hole diameter (threaded for an M8)
EM_Ax = math.pi / 4 * (M8.w_OD**2 - EM_holeD**2)    # mm2 - net area on EM




####### ALUMINUM COMPONENTS
Al_CTE = 23.6 * 10**(-6)        # C-1 - Linear CTE for Aluminum
Al_E = 68.9 * 10**3             # N/mm2 - Young's modulus of Aluminum

# 1) Aluminum busbar (normal)
Al_bus_L0 = 1.59                # mm - initial thickness



###### MEMBER STIFFNESS
alpha = 30 * math.pi / 180      # radians - 30 degrees in radians


(Will input a function for calculations at a later time, but first do this process for just ONE of the types!)

### Force Generated with Temperature Rise and Fall

In [24]:
def force_rise_fall(L0_Cu, Ax_Cu, bolt, L0_Al):
    # CALCULATION FOR BOLT
    L0_bolt = L0_Cu + L0_Al

    # THERMAL EXPANSION COEFFS
    Fcoeff_Cu = L0_Cu / (Ax_Cu * Cu_E)                      # mm/N
    Fcoeff_Al = L0_Al / (Ax_Cu * Al_E)                      # mm/N
    Fcoeff_bolt = L0_bolt / (bolt.Ax * bolt.E)                  # mm/N

    Tcoeff_Cu = L0_Cu * Cu_CTE * dT_max                     # mm
    Tcoeff_Al = L0_Al * Al_CTE * dT_max                     # mm
    Tcoeff_bolt = L0_bolt * bolt.CTE * dT_max                 # mm

    # TEMP RISE AND FALL CALCS
    Fcoeff_total = Fcoeff_Cu + Fcoeff_Al + Fcoeff_bolt      # the same for both rise and fall
    Tcoeff_temp = Tcoeff_Cu + Tcoeff_Al - Tcoeff_bolt       # temporary T coefficient
    Tcoeff_total = np.array([Tcoeff_temp, -Tcoeff_temp])    # entries for RISE (+), FALL (-) respectively

    F_result = Tcoeff_total / Fcoeff_total                  # N - resulting force

    print("Fcoeffs:",Fcoeff_Cu, Fcoeff_Al, Fcoeff_bolt)
    print("Tcoeffs:",Tcoeff_Cu, Tcoeff_Al, Tcoeff_bolt)
    print("Final Forces [N] (Rise, Fall):", F_result[0], F_result[1])

    return F_result

For lug/Al stackup:

In [25]:
F_lugs = force_rise_fall(lug_L0, lug_Ax, M6, Al_bus_L0)

Fcoeffs: 2.4611903161387836e-07 3.092889582352622e-07 6.383881606241581e-07
Tcoeffs: 0.00151803 0.00168858 0.0019981350000000003
Final Forces [N] (Rise, Fall): 1012.2959430895147 -1012.2959430895147


For EM/Al stackup:

In [26]:
F_EM = force_rise_fall(EM_L0, EM_Ax, M8, Al_bus_L0)

Fcoeffs: 4.822877063390768e-07 1.5303359912682245e-07 9.539349401570477e-07
Tcoeffs: 0.006012 0.00168858 0.005308065000000001
Final Forces [N] (Rise, Fall): 1505.4306104439415 -1505.4306104439415


### Member Stiffness

In [38]:
def member_stiffness(L0_Cu, Ax_Cu, bolt, L0_Al):
    ##### START FXN HERE
    total_t = L0_Cu + L0_Al + bolt.wt       # mm - total thickness
    half_t = total_t / 2                    # mm - half-way thickness

    # Note that this assumes that the washer is on the same side as the aluminum
    Al_test = half_t - bolt.wt - L0_Al
    if Al_test < 0:     # if Al thickness extends past half way
        top_half_Al = half_t - bolt.wt
        bot_half_Al = L0_Al - top_half_Al
        bot_half_Cu = L0_Cu
        top_half_Cu = 0
    else:               # if it doesn't
        top_half_Al = L0_Al
        bot_half_Al = 0
        bot_half_Cu = half_t
        top_half_Cu = L0_Cu - half_t

    thicknesses = np.array([top_half_Al, bot_half_Al, bot_half_Cu, top_half_Cu])    # mm - relative thickness in the member stiffness calcn
    index = np.where(thicknesses == 0)
    thicknesses = np.delete(thicknesses, index)
    E_modulus = np.delete(np.array([Al_E, Al_E, Cu_E, Cu_E]), index)

    # Pseudo washer diameter calc
    thinner_mat = min((bolt.wt + L0_Al), L0_Cu)                 # mm - thinner material
    pseudo_wd = bolt.w_OD + 2 * thinner_mat * math.tan(alpha)   # mm - pseudo washer diameter?

    # Calculations for member stiffness
    P = math.pi * E_modulus * bolt.d * math.tan(alpha)                                              # N/mm - temp value
    delta_num = (2 * thicknesses * math.tan(alpha) + bolt.w_OD - bolt.d) * (bolt.w_OD + bolt.d)     # temp value
    delta_denom = (2 * thicknesses * math.tan(alpha) + bolt.w_OD + bolt.d) * (bolt.w_OD - bolt.d)   # temp value
    k = P / np.log(delta_num / delta_denom)                                                         # N/mm - member stiffnesses

    combined_stiffness = (np.sum(1 / k))**(-1)  # N/mm - total member stiffness - summed as series springs

    return combined_stiffness

For Lug:

*# Set up parameters that will eventually be used in a function:
L0_Cu = lug_L0
Ax_Cu = lug_Ax
bolt = M6
L0_Al = Al_bus_L0

In [39]:
k_mem_lug = member_stiffness(lug_L0, lug_Ax, M6, Al_bus_L0)
k_mem_lug

2395658.42164211

For EM:

In [40]:
k_mem_EM = member_stiffness(EM_L0, EM_Ax, M8, Al_bus_L0)
k_mem_EM

2113149.487915191